In [1]:
'''
Objective:
•
Develop decision tree and random forest models with Sales as the target variable (first converted into a categorical variable) to identify the different attributes contributing to high sales for a cloth manufacturing company.
Constraints:
•
Ensure comprehensive preprocessing of the dataset, including handling missing values, encoding categorical variables, and balancing the target variable, to prepare the data for decision tree and random forest model training while maintaining data quality and integrity.
•
Employ robust evaluation techniques to assess the performance of the decision tree and random forest models, considering metrics such as accuracy, precision, recall, and F1-score, to ensure reliable insights into the attributes influencing high sales.
Benefits/Impact:
•
The decision tree and random forest models provide actionable insights into the attributes contributing to high sales for the cloth manufacturing company, enabling informed decision-making regarding product development, marketing strategies, and inventory management.
•
By leveraging insights from the models, the company can optimize resource allocation, tailor product offerings to customer preferences, and enhance sales performance, leading to increased revenue, market competitiveness, and customer satisfaction.

'''


import pandas as pd
import numpy as np
import sweetviz
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import joblib
import pickle

In [2]:
cloth = pd.read_csv('ClothCompany_Data.csv')


In [3]:
cloth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [4]:
cloth.isna().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

In [5]:
cloth_report = sweetviz.analyze([cloth, 'cloth'])
cloth_report.show_html('cloth_report.html')

                                             |                                                                …

Report cloth_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [6]:
x = cloth.drop(['Sales','Education','US'], axis = 1)
y = cloth['Sales']

In [7]:
numeric_features = x.select_dtypes(exclude = ['object']).columns
numeric_features

Index(['CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age'], dtype='object')

In [8]:
categorical_features = x.select_dtypes(include = ['object']).columns
categorical_features

Index(['ShelveLoc', 'Urban'], dtype='object')

In [9]:
#imputation

In [10]:
num_pipeline = Pipeline(steps = [('impute', SimpleImputer(strategy = 'mean'))])

In [11]:
scale_pipeline = Pipeline(steps =[('scale', MinMaxScaler())])

#onehotencoding

In [12]:
encode_pipeline = Pipeline(steps = [('onehot', OneHotEncoder(drop = 'first'))])
encode_preprocessor = ColumnTransformer(transformers=[
    ('encode', encode_pipeline, categorical_features)
])
encode_data =encode_preprocessor.fit_transform(x)
cate = pd.DataFrame(encode_data, columns = list(encode_preprocessor.get_feature_names_out()))
cate

encode__ShelveLoc_Good  encode__ShelveLoc_Medium  encode__Urban_Yes
0                       0.0                       0.0                1.0
1                       1.0                       0.0                1.0
2                       0.0                       1.0                1.0
3                       0.0                       1.0                1.0
4                       0.0                       0.0                1.0
..                      ...                       ...                ...
395                     1.0                       0.0                1.0
396                     0.0                       1.0                0.0
397                     0.0                       1.0                1.0
398                     0.0                       0.0                1.0
399                     1.0                       0.0                1.0

[400 rows x 3 columns]

In [13]:
preprocessor = ColumnTransformer(transformers = [
    ('num', num_pipeline, numeric_features),
])

In [14]:

imp_enc_scale = preprocessor.fit_transform(x)

In [15]:
cleandata = pd.DataFrame(imp_enc_scale, columns = list(preprocessor.get_feature_names_out()))
cleandata

num__CompPrice  num__Income  num__Advertising  num__Population  \
0             138.0         73.0              11.0            276.0   
1             111.0         48.0              16.0            260.0   
2             113.0         35.0              10.0            269.0   
3             117.0        100.0               4.0            466.0   
4             141.0         64.0               3.0            340.0   
..              ...          ...               ...              ...   
395           138.0        108.0              17.0            203.0   
396           139.0         23.0               3.0             37.0   
397           162.0         26.0              12.0            368.0   
398           100.0         79.0               7.0            284.0   
399           134.0         37.0               0.0             27.0   

     num__Price  num__Age  
0         120.0      42.0  
1          83.0      65.0  
2          80.0      59.0  
3          97.0      55.0  
4         128.0      38.0  
..          ...       ...  
395       128.0      33.0  
396       120.0      55.0  
397       159.0      40.0  
398        95.0      50.0  
399       120.0      49.0  

[400 rows x 6 columns]

In [16]:
scale_preprocessor = ColumnTransformer(transformers =[('scale', scale_pipeline, cleandata.columns)])



scale_data = scale_preprocessor.fit_transform(cleandata)



scale_df = pd.DataFrame(scale_data, columns = list(scale_preprocessor.get_feature_names_out()))
joblib.dump(imp_enc_scale, 'decision_tree_assignment')

['decision_tree_assignment']

In [17]:
wins = Winsorizer(capping_method = 'iqr',
                 tail = 'both',
                 fold = 1.5,
                 variables = list(scale_df.columns))

In [18]:

outlier_pipeline = Pipeline([('wins', wins)])

In [19]:
outlier_data = outlier_pipeline.fit_transform(scale_df)

In [20]:
joblib.dump(outlier_data, 'decision_tree_wins')

['decision_tree_wins']

In [21]:
cleandata1 = pd.concat([outlier_data, cate])
cleandata1

scale__num__CompPrice  scale__num__Income  scale__num__Advertising  \
0                 0.622449            0.525253                 0.379310   
1                 0.346939            0.272727                 0.551724   
2                 0.367347            0.141414                 0.344828   
3                 0.408163            0.797980                 0.137931   
4                 0.653061            0.434343                 0.103448   
..                     ...                 ...                      ...   
395                    NaN                 NaN                      NaN   
396                    NaN                 NaN                      NaN   
397                    NaN                 NaN                      NaN   
398                    NaN                 NaN                      NaN   
399                    NaN                 NaN                      NaN   

     scale__num__Population  scale__num__Price  scale__num__Age  \
0                  0.533066           0.574850         0.309091   
1                  0.501002           0.353293         0.727273   
2                  0.519038           0.335329         0.618182   
3                  0.913828           0.437126         0.545455   
4                  0.661323           0.622754         0.236364   
..                      ...                ...              ...   
395                     NaN                NaN              NaN   
396                     NaN                NaN              NaN   
397                     NaN                NaN              NaN   
398                     NaN                NaN              NaN   
399                     NaN                NaN              NaN   

     encode__ShelveLoc_Good  encode__ShelveLoc_Medium  encode__Urban_Yes  
0                       NaN                       NaN                NaN  
1                       NaN                       NaN                NaN  
2                       NaN                       NaN                NaN  
3                       NaN                       NaN                NaN  
4                       NaN                       NaN                NaN  
..                      ...                       ...                ...  
395                     1.0                       0.0                1.0  
396                     0.0                       1.0                0.0  
397                     0.0                       1.0                1.0  
398                     0.0                       0.0                1.0  
399                     1.0                       0.0                1.0  

[800 rows x 9 columns]

In [22]:
#converting target variable numeric into categorical 

In [23]:
bins = [0, 5, 10, 15, float('inf')]
labels = ['Low','medium','high','very high']

In [24]:
y1 = pd.DataFrame(y)

In [25]:
y1['categorical_sales'] = pd.cut(y1['Sales'],bins = bins, labels = labels)

In [26]:
y1

Sales categorical_sales
0     9.50            medium
1    11.22              high
2    10.06              high
3     7.40            medium
4     4.15               Low
..     ...               ...
395  12.57              high
396   6.14            medium
397   7.41            medium
398   5.94            medium
399   9.71            medium

[400 rows x 2 columns]

In [27]:
#data split
y1 = y1.fillna(y1.mode().iloc[0])


In [28]:
x_train, x_test, y_train, y_test = train_test_split(cleandata, y, test_size = 0.2, random_state = 0)

In [29]:
##model building
x_train1 = x_train.astype('int')
x_test1 = x_test.astype('int')
y_train1 = y_train.astype('int')
y_test1 = y_test.astype('int')

In [30]:
model =dt(criterion = 'entropy')

In [31]:
model.fit(x_train1, y_train1)

DecisionTreeClassifier(criterion='entropy')

In [32]:
pred = model.predict(x_test1)

In [33]:
pred

array([ 5,  7,  2,  5, 11, 12,  9,  6,  8,  6,  5,  2,  7, 10,  4,  6,  7,
       11,  7,  6,  5,  8,  4,  5,  2,  3,  4,  3,  6,  5,  8, 12,  9,  7,
        7,  7,  4, 11,  6,  5,  6,  4,  5,  8,  6,  4, 11, 10,  5,  7,  3,
        6,  6,  3,  5,  5,  5, 12,  8,  8, 11,  7,  4,  4,  5,  5, 11, 11,
       11,  7,  8, 12,  5,  4,  6,  8,  5,  5, 12,  6])

In [34]:
accuracy_score(y_test1, pred)*100

10.0

In [35]:
pd.crosstab(y_test1, pred, rownames = ['actual'], colnames = ['prediction'])

prediction  2   3   4   5   6   7   8   9   10  11  12
actual                                                
2            0   0   0   1   0   0   0   0   0   0   0
3            1   0   1   0   1   0   0   0   0   0   0
4            0   0   3   2   2   2   1   0   0   0   0
5            0   1   2   2   2   0   0   0   0   0   0
6            0   0   1   2   1   2   2   0   0   5   0
7            2   0   0   1   2   0   0   2   1   0   1
8            0   1   1   1   1   1   2   0   0   1   2
9            0   1   1   3   1   1   0   0   0   1   1
10           0   0   0   2   0   1   0   0   0   1   1
11           0   0   0   1   1   2   3   0   0   0   0
12           0   1   0   1   1   1   0   0   1   0   0
13           0   0   0   1   0   0   0   0   0   0   0

In [36]:
### Hyperparameter Optimization


In [37]:
param_grid = {'criterion': ['gini','entropy'], 'max_depth': np.arange(3,15)}

In [38]:
tree_model = dt()

In [39]:
tree_grid = GridSearchCV(tree_model, param_grid, cv = 5, scoring = 'accuracy', return_train_score = False, verbose = 1)

In [40]:
tree_grid.fit(x_train1, y_train1)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
             scoring='accuracy', verbose=1)

In [41]:
pred1 =tree_grid.predict(x_test1)

In [42]:
accuracy_score(y_test1, pred1)*100

15.0

In [43]:
tree_grid.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [44]:
tree_grid.best_params_

{'criterion': 'entropy', 'max_depth': 4}

In [45]:
model1 = dt(criterion ='gini', max_depth = 4)

In [46]:
model1.fit(x_train1, y_train1)

DecisionTreeClassifier(max_depth=4)

In [47]:
pred2 = model1.predict(x_test1)

In [48]:
accuracy_score(y_test1, pred2)*100

11.25

In [49]:
from sklearn.ensemble import RandomForestClassifier as rfc

In [50]:
rf = rfc()

In [51]:
rf.fit(x_train1, y_train1)

RandomForestClassifier()

In [52]:
rf_pred = rf.predict(x_test1)

In [53]:
accuracy_score(y_test1, rf_pred)

0.0875

In [54]:
#parameter tuning in randomforest

In [55]:
param_grid1 ={
    'n_estimators': [50,100,150], 'max_depth': [None, 10, 20], 'min_samples_split': [2,5,9], 'min_samples_leaf': [1,3,5], 'max_features': ['auto','sqrt','log2']
}

In [56]:
rf_model = rfc()

In [57]:
rf_grid = GridSearchCV(estimator = rf_model, param_grid = param_grid1, cv = 5, scoring = 'accuracy')

In [58]:
rf_grid.fit(x_train1, y_train1)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._valida

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [2, 5, 9],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy')

In [59]:
rf_grid.best_params_

{'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 50}

In [60]:
rf_grid.best_estimator_

RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=50)

In [61]:
rf_grid.best_score_

0.18125

In [62]:
accuracy_score(y_test1, rf_grid.predict(x_test1))

0.1

In [69]:
rf1_model = rfc(max_features = 'sqrt', max_depth = 10, min_samples_split = 5, min_samples_leaf = 1, n_estimators = 50)

In [70]:
rf1_model.fit(x_train1, y_train1)

RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=50)

In [71]:
prediction_rf = rf1_model.predict(x_test1)
prediction_rf

array([ 4, 10,  7,  5,  8, 10,  9,  6,  8,  6,  4,  6,  8, 10,  7,  5,  7,
        9,  5,  7,  5,  8,  4,  5,  5,  4,  4,  4,  4,  6,  9,  9,  8,  7,
        7,  8,  7,  7,  6,  5,  6,  5,  6, 10,  5,  2,  9,  8,  8, 12,  7,
        8,  8,  8,  5,  9,  8,  8,  6,  5,  6,  5,  7,  5,  6,  8,  9,  8,
        9,  8,  9, 10,  6,  7,  6,  6,  5,  5,  7,  9])

In [72]:
accuracy_score(y_test1, prediction_rf)

0.125